In [428]:
import requests
import xml.etree.ElementTree as ET
import time
import json
import difflib
from collections import namedtuple
from urllib.parse import quote, unquote
from os.path import isfile
from os.path import split as splitPath
from os.path import join as joinPath
from openpyxl import Workbook
from random import random

In [432]:
Request_info = namedtuple("Request_info", ["URL", "headers", "postData"])
Item_info = namedtuple("Item_info", ["name", "code"])
Request_param = namedtuple("Request_param", ["category", 
                                             "sWD_ReportId", "sWD_TableId", 
                                             "sheet_name", "sheet_title"])

# Global Variable
GET_DATA_INFO_PATH = "./config/getData.json"
SUPPORT_ITEM = ["economy", "product", "partner"]

# The following path must be this form
GET_ITEM_TEMPLATE = "./config/get{}.json"
def default_get_item_path(itemName):
    if(itemName.lower() not in SUPPORT_ITEM):
        raise ValueError("[!] This item is not supported. ")
    return GET_ITEM_TEMPLATE.format(itemName.capitalize())

# itemCode info path
ITEM_CODE_PATH_TEMPLATE = "./config/{}Code.json"
def default_itemCode_path(itemName):
    if(itemName.lower() not in SUPPORT_ITEM):
        raise ValueError("[!] This item is not supported. ")
    return ITEM_CODE_PATH_TEMPLATE.format(itemName)

# request_info
REQUEST_PARAM = dict()
REQUEST_PARAM["export"] = Request_param(category="export",
                                        sWD_ReportId='182117', 
                                        sWD_TableId='1495', 
                                        sheet_name="科技產品出口",
                                        sheet_title="Merchandise trade matrix – " + \
                                                    "product groups, exports in thousands " + \
                                                    "of United States dollars, ")

REQUEST_PARAM["import"] = Request_param(category="import", 
                                        sWD_ReportId='180622', 
                                        sWD_TableId='1494', 
                                        sheet_name="科技產品進口", 
                                        sheet_title="Merchandise trade matrix – " + \
                                                    "product groups, imports in thousands " + \
                                                    "of United States dollars, ")

# supported data
CATEGORY = dict()
CATEGORY["import"] = (REQUEST_PARAM["import"])
CATEGORY["export"] = (REQUEST_PARAM["export"])
CATEGORY["both"] = (REQUEST_PARAM["export"], REQUEST_PARAM["import"])

In [477]:
class ItemCode_Getter:
    # This is the tool, which is used to obtain the code of economy, partner or product
    def __init__(self, whatData:str, getItem_info_path = None):
        self.itemCode = dict()
        self.whatData = whatData
        self.default_getItem_info = getItem_info_path
        
        if(not self.default_getItem_info):
            try:
                self.itemCode = json.load(open(default_itemCode_path(whatData), "r", encoding="utf-8"))
            except FileNotFoundError:
                self.update()
                self.save()
        elif(not isfile(getItem_info_path)):
            self.default_getItem_info = None
            try:
                self.itemCode = json.load(open(default_itemCode_path(whatData), "r", encoding="utf-8"))
            except FileNotFoundError:
                self.update()
                self.save()
        elif(isfile(getItem_info_path)):
            self.itemCode = json.load(open(self.default_getItem_info, "r", encoding="utf-8").read())
    def save(self, output_path = None):
        if(not output_path):
            output_path = default_itemCode_path(self.whatData)
            
        with open(output_path, "w") as file:
            file.write(json.dumps(self.itemCode))
    
    def _request_itemCode(self, request_info_path):
        # read the request "getItem" info
        rq_info = str() # request info
        with open(request_info_path, "r", encoding = "utf-8") as file:
            rq_info = file.read()
            rq_info = json.loads(rq_info)

        # extract data from the rq_info
        # deal with get method
        URL = rq_info["url"].split('?')[0] # unfinished
        queryString = dict()
        for data in rq_info["queryString"]:
            queryString[data["name"]] = data["value"]
        
        queryString["rowCount"] = str(1000000)
        # rowCount control how many return values we can get
        # if we set an extreme value, we can get all data
        
        for i, key in enumerate(queryString.keys()):
            if(i == 0):
                URL = URL + '?'
            else:
                URL = URL + '&'
            URL = URL + key + '=' + queryString[key]
        
        # deal with headers
        headers = dict()
        for data in rq_info["headers"]:
            headers[data["name"]] = data["value"]

        # deal with post method
        postData = rq_info["postData"]["params"]
        postDataStr = ""
        if(isinstance(postData, list)):
            for i, post in enumerate(postData):
                if(i != 0):
                    postDataStr = postDataStr + '&'
                postDataStr = postDataStr + post["name"] + '=' + post["value"]
            postData = postDataStr
        
        # request itemCode
        itemGetter = requests.post(url = URL, headers = headers, data = postData)
        return itemGetter.text
            
    def update(self, request_info_path=None, save = False,
               output_path=None):
        if(not request_info_path):
            request_info_path = default_get_item_path(self.whatData)
        if(not output_path):
            output_path = default_itemCode_path(self.whatData)
        
        itemCode_xml = self._request_itemCode(request_info_path)
        root = ET.fromstring(itemCode_xml)
        for child in root:
            data = child.attrib
            self.itemCode[data["Label"].lower().strip()] = \
                    Item_info(name=data["Label"], code=data["Handle"])
        if(save):
            self.save(output_path)
        
    def search(self, item):
        if(isinstance(item, str)):
            return self.itemCode[item.lower()]
        
        # if item is of iterable type such as tuple or list
        itemCode = []
        for i in iter(item):
            itemCode.append(self.itemCode[i.lower()])
        return tuple(itemCode)
    
class UNCTADSTAT_parser:
# 可以用recursive處理economy的多種輸入    
    def __init__(self, delay_max = 3, update_code = True):
        
        self.delay_max = delay_max
                
        self.economyCode = ItemCode_Getter("economy")        
        self.productCode = ItemCode_Getter("product")
        self.partnerCode = ItemCode_Getter("partner")
        
        if(update_code):
            self.update_all_code()
        
        self.rq_info = None
        self._read_request_info()# assign value to self.rq_info
        
    def _read_request_info(self, rq_info_path=GET_DATA_INFO_PATH):
        # read the request "getData" info
        rq_info = str() # request info
        with open(rq_info_path, "r", encoding = "utf-8") as file:
            rq_info = file.read()
            rq_info = json.loads(rq_info)

        # extract data from the rq_info
        URL = rq_info["url"]
        headers = dict()
        for data in rq_info["headers"]:
            headers[data["name"]] = data["value"]

        postData = dict()
        for i, post in enumerate(rq_info["postData"]["params"]):
            postData[post["name"]] = post["value"]
            
        self.rq_info = Request_info(URL=URL, headers=headers, postData=postData)
        
    def _adjust_postData(self, request_param, economy:str, product:str, partner:str):
        # The transmission from str to code occurs in this function
        postData = self.rq_info.postData
        reportView = unquote(postData["sWD_ReportView"])
        reportView = '<?xml version="1.0" encoding="UTF-8"?>' + reportView
        reportView = reportView.replace('+', ' ') 
        reportView = ET.fromstring(reportView)
        
        # request specific data
        postData["sWD_ReportId"] = request_param.sWD_ReportId
        postData["sWD_TableId"] = request_param.sWD_TableId
        
        # adjust economy
        for target in reportView.findall(r"./RowDims/Dim[@name='ECONOMY']"):
            target.find(".//String").attrib["value"] = economy # only one each request
        
        # adjust product
        for target in reportView.findall(r"./OtherDims/Dim[@name='PRODUCT']"):
            target.find(".//ActiveItem").attrib["pos"] = product # only one each request
        
        # adjust partner
        for target in reportView.findall(r"./OtherDims/Dim[@name='PARTNER']"):
            target.find(".//String").attrib["value"] = partner # only one each request
        
        reportView = ET.tostring(reportView).decode("utf-8")
        postData["sWD_ReportView"] = reportView
        postDataStr = ""
        count = 0
        for key, value in postData.items():
            if(count != 0):
                postDataStr = postDataStr + '&'
            postDataStr = postDataStr + key + '=' + value
            count += 1
        return postDataStr
        
    def _specific_request(self, request_param, economy:str, product:str, partner:str):
        postData = self._adjust_postData(request_param, economy, product, partner)
        dataGetter = requests.post(url=self.rq_info.URL, headers=self.rq_info.headers, data=postData)
        target = list()
        result = ET.fromstring(dataGetter.text)
        for value in result.findall(".//C"):
            try:
                target.append(int("".join(value.attrib["f"].split()))) # get rid of space
            except ValueError:
                target.append(value.attrib["f"]) # deal with NA
        return target
        
    def _save_one_sheet(self, ws:openpyxl.worksheet.worksheet.Worksheet, 
                        request_param:Request_param, 
                        economy, Product, Partner, Period, 
                        display_progress_rate=True):
        
        # economy, time, partner, product_group
        PADDING = ' ' * 10
        ws.title = request_param.sheet_name
        ws["A1"] = request_param.sheet_title
        ws["A4"] = "ECONOMY"
        ws["A6"] = "YEAR"
        ws["A7"] = "PRODUCT"
        ws["B7"] = "PARTNER"
        
        line = 8
        
        ws["B4"] = economy.name
        for product in Product:
            cellColumn = ord('C')
            ws["A{}".format(line)] = product.name
            for partner in Partner:
                ws["B{}".format(line)] = partner.name
                target = self._specific_request(request_param, economy.code, product.code, partner.code)
                
                # deal with the fixed year period
                if(line == 8):
                    startYear = 1995
                    ws["A1"] = ws["A1"].value + "{}-{}".format(startYear, startYear+len(target)-1)
                    for i in range(len(target)):
                        ws.cell(row=6, column=3+i, value=str(startYear+i))
                
                for i in range(len(target)):
                    ws["{}{}".format(chr(cellColumn+i), line)] = target[i]
                    time.sleep(self.delay_max * random())
                line += 1
            if(display_progress_rate):
                print("Finish saving table: category-\"{}\", economy-\"{}\", product-\"{}\"".
                      format(request_param.sheet_name, economy.name.strip(), product.name.strip()))
    
    def request_and_save(self, whatData, Economy, Product, Partner,
                        output_path, Period = None, display_progress_rate=True):
        
        if(not whatData not in SUPPORT_ITEM):
            raise ValueError("[!] The data you request is not supported.  ")

        # This block may move to _save_one_ws to get more flexibility in the fulture, 
        # though put it here can increase some efficiency
        Economy = tuple([self.economyCode.search(i) for i in Economy])
        Product = tuple([self.productCode.search(i) for i in Product])
        Partner = tuple([self.partnerCode.search(i) for i in Partner])
        # ---
        
        category = CATEGORY[whatData.lower()]
        for economy in Economy:
            wb = Workbook()
            ws = wb.active
            for i, param in enumerate(category):
                if(i != 0):
                    ws = wb.create_sheet()
                self._save_one_sheet(ws, param, economy, Product, Partner, Period, 
                                     display_progress_rate=display_progress_rate)
            
            adjust_output_path = output_path
            if(len(Economy) != 1):
                folder, file = splitPath(output_path)
                fileName, ext = file.split('.')
                adjust_fileName = "{}_{}".format(fileName, economy.name.strip().replace(' ', '_'))
                adjust_output_path = joinPath(folder, "{}.{}".format(adjust_fileName, ext))
            
            wb.save(adjust_output_path)
            if(display_progress_rate):
                print("Finish saving the data of economy:{}. ".format(economy.name.strip()))
    
    def update_economy(self):
        self.economyCode.update()
    
    def update_product(self):
        self.productCode.update()
    
    def update_partner(self):
        self.partnerCode.update()
    
    def update_all_code(self):
        self.update_economy()
        time.sleep(random() * self.delay_max)
        
        self.update_product()
        time.sleep(random() * self.delay_max)
        
        self.update_partner()
        time.sleep(random() * self.delay_max)
        
    def save_all_code(self):
        self.economyCode.save()
        self.productCode.save()
        self.partnerCode.save()

In [478]:
parser = UNCTADSTAT_parser(delay_max = 1)

In [479]:
Economy = ["viet nam"]
Product = ["total all products"]
Partner = ["china"]
output_file_name = "./result.xlsx"

parser.request_and_save("both", Economy, Product, Partner, output_file_name)

Finish saving table: category-"科技產品出口", economy-"Viet Nam", product-"Total all products"
Finish saving table: category-"科技產品進口", economy-"Viet Nam", product-"Total all products"
Finish saving the data of economy:Viet Nam. 


In [419]:
parser.update_all_code()
parser.save_all_code()

In [286]:
Economy = ["viet nam"]
Product = ["total all products", "Manufactured goods (SITC 5 to 8 less 667 and 68)", 
            "Machinery and transport equipment (SITC 7)"]
Partner = ["china", "China, Taiwan Province of", "japan", "Korea, Republic of",
           "United States of America", "ASEAN (Association of Southeast Asian Nations)",
           "EU28 (European Union)"]
output_file_name = "./haha.xlsx"
        Electronic excluding parts and components (SITC 751 + 752 + 761 + 762 + 763 + 775)
        Parts and components for electrical and electronic goods (SITC 759 + 764 + 772 +776)
        Other machinery and transport equipment (SITC 7 - (751 + 752 + 761 + 762 + 763 + 775 + 759 + 764 + 772 + 776))
      Other manufactured goods (SITC 6 + 8 less 667 and 68)
        Textile fibres, yarn, fabrics and clothing (SITC 26 + 65 + 84)
Manufactured goods by degree of manufacturing
  Labour-intensive and resource-intensive manufactures
  Low-skill and technology-intensive manufactures
  Medium-skill and technology-intensive manufactures
    Medium-skill: Electronics (excluding parts and components) (SITC 775)
    Medium-skill: Parts and components for electrical and electronic goods (SITC 772)
    Medium-skill: Other, excluding electronics
  High-skill and technology-intensive manufactures
    High-skill: Electronics (excluding parts and components) (SITC 751 + 752 + 761 + 762 + 763)
    High-skill: Parts and components for electrical and electronic goods (SITC 759 + 764 + 776)
    High-skill: Other, excluding electronics

parser.request_and_save(Economy, Product, Partner, output_file_name)

In [271]:
# print(ItemCode_Getter.default_getItem_info)
from os import listdir
itemGetter = ItemCode_Getter("economy")

In [252]:
# itemGetter.save("./config/partnerCode.json")

In [253]:
itemGetter.search(["japan"])

KeyError: 'japan'

In [272]:
print(len(itemGetter.itemCode))
itemGetter.itemCode

357


{'individual economies': '0',
 'afghanistan': '1',
 'albania': '2',
 'algeria': '3',
 'american samoa': '4',
 'andorra': '5',
 'angola': '6',
 'anguilla': '7',
 'antigua and barbuda': '8',
 'argentina': '9',
 'armenia': '10',
 'aruba': '11',
 'australia': '12',
 'austria': '13',
 'azerbaijan': '14',
 'bahamas': '15',
 'bahrain': '16',
 'bangladesh': '17',
 'barbados': '18',
 'belarus': '19',
 'belgium': '20',
 'belize': '21',
 'benin': '22',
 'bermuda': '23',
 'bhutan': '24',
 'bolivia (plurinational state of)': '25',
 'bonaire, sint eustatius and saba': '26',
 'bosnia and herzegovina': '27',
 'botswana': '28',
 'brazil': '29',
 'british virgin islands': '30',
 'brunei darussalam': '31',
 'bulgaria': '32',
 'burkina faso': '33',
 'burundi': '34',
 'cabo verde': '35',
 'cambodia': '36',
 'cameroon': '37',
 'canada': '38',
 'cayman islands': '39',
 'central african republic': '40',
 'chad': '41',
 'chile': '42',
 'china': '43',
 'china, hong kong sar': '44',
 'china, macao sar': '45',
 '